# E07 recommendation project

## 1) 데이터 준비와 전처리

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/e/e07_recommendation_artist/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'cnt'}, inplace=True)

In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/e/e07_recommendation_artist/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings

,user_id,movie_id,cnt,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [6]:
movies

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [7]:
data = pd.merge(ratings, movies, how='outer', on='movie_id')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836733 entries, 0 to 836732
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    836478 non-null  float64
 1   movie_id   836733 non-null  int64  
 2   cnt        836478 non-null  float64
 3   timestamp  836478 non-null  float64
 4   title      836733 non-null  object 
 5   genre      836733 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 44.7+ MB


In [8]:
# null 다 떨굼

In [9]:
data = data.drop(columns=['timestamp']).dropna()

In [10]:
data

,user_id,movie_id,cnt,title,genre
0,1.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama
1,2.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama
2,12.0,1193,4.0,One Flew Over the Cuckoo's Nest (1975),Drama
3,15.0,1193,4.0,One Flew Over the Cuckoo's Nest (1975),Drama
4,17.0,1193,5.0,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836473,5851.0,3607,5.0,One Little Indian (1973),Comedy|Drama|Western
836474,5854.0,3026,4.0,Slaughterhouse (1987),Horror
836475,5854.0,690,3.0,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938.0,2909,4.0,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [11]:
# float -> int, timestamp column을 drop한다
data['user_id'] = data['user_id'].astype(int)
data['cnt'] = data['cnt'].astype(int)
data

,user_id,movie_id,cnt,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836473,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,Slaughterhouse (1987),Horror
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [12]:
data['user_id'].nunique()
data['user_id'].max()

6040

In [13]:
ratings_origin = ratings
ratings = data

In [14]:
ratings['user_id'] = ratings['user_id'].astype(int)
ratings['cnt'] = ratings['cnt'].astype(int)
ratings['movie_id'] = ratings['movie_id'].astype(int)
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836478 entries, 0 to 836477
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   836478 non-null  int64 
 1   movie_id  836478 non-null  int64 
 2   cnt       836478 non-null  int64 
 3   title     836478 non-null  object
 4   genre     836478 non-null  object
dtypes: int64(3), object(2)
memory usage: 38.3+ MB


## 2) 분석하기
---
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기있는 영화 30개(인기순)

In [15]:
print("unique movie title number:", ratings['movie_id'].nunique())
print("unique user number:", ratings['user_id'].nunique())
print("-ntop 30 popular movies/n", ratings['title'].value_counts().head(30))

unique movie title number: 3628
unique user number: 6039
-ntop 30 popular movies/n American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                              

In [16]:
midnew = ratings['movie_id'].max()+1
uid=ratings['user_id'].max()+1
print(midnew, uid)

3953 6041


## 4) CSR matrix를 직접 만들어 봅시다.

In [17]:
mydat = pd.DataFrame(columns=['user_id','movie_id','cnt','title','genre'])
mydat.loc[0] = [6041, 318, 5, 'Shawshank Redemption, The (1994)', 'Drama']
mydat.loc[1] = [uid, midnew, 5, "8 mile (2003)", "Drama"]
mydat.loc[2] = [uid, 1270, 4, "Back to the Future (1985)", "Comedy|Sci-Fi"]
midnew += 1
mydat.loc[3] = [uid, midnew, 4, "Inception (2010)", "SF|Action"]
mydat.loc[4] = [uid, 593, 5, "Silence of the Lambs, The (1991)", "Drama|Thriller"]

mydat

,user_id,movie_id,cnt,title,genre
0,6041,318,5,"Shawshank Redemption, The (1994)",Drama
1,6041,3953,5,8 mile (2003),Drama
2,6041,1270,4,Back to the Future (1985),Comedy|Sci-Fi
3,6041,3954,4,Inception (2010),SF|Action
4,6041,593,5,"Silence of the Lambs, The (1991)",Drama|Thriller


In [18]:
ratings = ratings.append(mydat)
ratings

,user_id,movie_id,cnt,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
0,6041,318,5,"Shawshank Redemption, The (1994)",Drama
1,6041,3953,5,8 mile (2003),Drama
2,6041,1270,4,Back to the Future (1985),Comedy|Sci-Fi
3,6041,3954,4,Inception (2010),SF|Action


In [19]:
ratings['user_id'] = ratings['user_id'].astype(int)
ratings['cnt'] = ratings['cnt'].astype(int)
ratings['movie_id'] = ratings['movie_id'].astype(int)
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836483 entries, 0 to 4
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   836483 non-null  int64 
 1   movie_id  836483 non-null  int64 
 2   cnt       836483 non-null  int64 
 3   title     836483 non-null  object
 4   genre     836483 non-null  object
dtypes: int64(3), object(2)
memory usage: 38.3+ MB


In [20]:
ratings[ratings['user_id']==6041]

,user_id,movie_id,cnt,title,genre
0,6041,318,5,"Shawshank Redemption, The (1994)",Drama
1,6041,3953,5,8 mile (2003),Drama
2,6041,1270,4,Back to the Future (1985),Comedy|Sci-Fi
3,6041,3954,4,Inception (2010),SF|Action
4,6041,593,5,"Silence of the Lambs, The (1991)",Drama|Thriller


In [21]:
user_unique = ratings['user_id'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}


In [22]:
movie_id_unique = ratings['movie_id'].unique()

movie_id_to_idx = {v:k for k,v in enumerate(movie_id_unique)}


In [23]:
title_unique = ratings['title'].unique()

title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [24]:
print(user_to_idx[100])
print(movie_id_to_idx[1])
print(title_to_idx["Back to the Future (1985)"])

3507
40
22


In [25]:
user_to_idx

{1: 0,
 2: 1,
 12: 2,
 15: 3,
 17: 4,
 18: 5,
 19: 6,
 24: 7,
 28: 8,
 33: 9,
 39: 10,
 42: 11,
 44: 12,
 47: 13,
 48: 14,
 49: 15,
 53: 16,
 54: 17,
 58: 18,
 59: 19,
 62: 20,
 80: 21,
 81: 22,
 88: 23,
 89: 24,
 95: 25,
 96: 26,
 102: 27,
 107: 28,
 120: 29,
 123: 30,
 131: 31,
 136: 32,
 139: 33,
 146: 34,
 150: 35,
 151: 36,
 155: 37,
 166: 38,
 169: 39,
 175: 40,
 178: 41,
 180: 42,
 190: 43,
 191: 44,
 192: 45,
 195: 46,
 196: 47,
 202: 48,
 208: 49,
 216: 50,
 224: 51,
 225: 52,
 227: 53,
 230: 54,
 231: 55,
 235: 56,
 236: 57,
 242: 58,
 245: 59,
 259: 60,
 261: 61,
 264: 62,
 265: 63,
 266: 64,
 270: 65,
 272: 66,
 281: 67,
 285: 68,
 287: 69,
 291: 70,
 292: 71,
 293: 72,
 294: 73,
 295: 74,
 297: 75,
 299: 76,
 300: 77,
 301: 78,
 306: 79,
 307: 80,
 308: 81,
 309: 82,
 312: 83,
 314: 84,
 315: 85,
 317: 86,
 318: 87,
 319: 88,
 326: 89,
 327: 90,
 329: 91,
 331: 92,
 332: 93,
 333: 94,
 334: 95,
 335: 96,
 339: 97,
 340: 98,
 349: 99,
 352: 100,
 353: 101,
 355: 102,
 357: 

In [26]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()

print("t_u_d length:", len(temp_user_data))
print("uid length:", len(ratings))

if len(temp_user_data) == len(ratings):
    print("user id col indexing OK!!")
    ratings['user_id'] = temp_user_data
else:
    print("user_id col indexing FAIL!!")


temp_title_data = ratings['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(ratings):
    print("title col indexing OK!!")
    ratings['title'] = temp_title_data
else:
    print("title col indexing FAIL!!")


ratings

t_u_d length: 836483
uid length: 836483
user id col indexing OK!!
title col indexing OK!!


,user_id,movie_id,cnt,title,genre
0,0,1193,5,0,Drama
1,1,1193,5,0,Drama
2,2,1193,4,0,Drama
3,3,1193,4,0,Drama
4,4,1193,5,0,Drama
...,...,...,...,...,...
0,6039,318,5,157,Drama
1,6039,3953,5,3628,Drama
2,6039,1270,4,22,Comedy|Sci-Fi
3,6039,3954,4,3629,SF|Action


## CSR Matrix


In [27]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_title = ratings['title'].nunique()

csr_data = csr_matrix((ratings.cnt, (ratings.user_id, ratings.title)), shape=(num_user, num_title))

In [28]:
csr_data

<6040x3630 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.
## MF Model 학습하기

In [29]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [30]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3630x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [31]:
# 모델 훈련
als_model.fit(csr_data_transpose)

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악

In [32]:
myid, shawshank = user_to_idx[6041], title_to_idx["Shawshank Redemption, The (1994)"]
myid_vec, shawshank_vec = als_model.user_factors[myid], als_model.item_factors[shawshank]

In [33]:
shawshank_vec

array([ 0.00361633, -0.02260149, -0.02347906,  0.03884713, -0.02859312,
        0.02317532,  0.00076774,  0.01183775,  0.01520667, -0.00647727,
        0.0251879 , -0.01277295,  0.00766861, -0.01337628, -0.00813822,
        0.05085362, -0.003747  , -0.0178614 , -0.03530301,  0.01855783,
        0.02082657, -0.00658514,  0.01620367,  0.01082794,  0.01824498,
        0.03274823,  0.000861  , -0.01308314, -0.01504275,  0.01693779,
        0.0309313 ,  0.00789163, -0.02695516,  0.02952161,  0.01385464,
        0.00188465, -0.00463441, -0.01497131, -0.00466755,  0.01052724,
       -0.01279105,  0.02044004,  0.01045301,  0.00314488,  0.0281276 ,
        0.01197921,  0.00474538, -0.00698844,  0.00587526, -0.01423356,
        0.00754628, -0.013164  ,  0.00571474, -0.00191561,  0.01747607,
        0.01678737, -0.02239294,  0.01288637,  0.0087376 ,  0.00107139,
       -0.00057612,  0.00905071,  0.01921833, -0.00833583,  0.03781997,
        0.00074045, -0.0059992 ,  0.0150693 ,  0.00339563, -0.00

In [34]:
np.dot(myid_vec, shawshank_vec)

0.57957864

In [35]:
tmp = data[data['title'].str.contains("Star Wars")]
print(tmp['title'].unique())

['Star Wars: Episode IV - A New Hope (1977)'
 'Star Wars: Episode I - The Phantom Menace (1999)'
 'Star Wars: Episode VI - Return of the Jedi (1983)'
 'Star Wars: Episode V - The Empire Strikes Back (1980)']


## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다

In [36]:
# 다른 영화에 대한 선호도 예측 확인하기

sixsense = title_to_idx['Sixth Sense, The (1999)']
sixsense_vec = als_model.item_factors[sixsense]
np.dot(myid_vec, sixsense_vec)

# 안 좋아하는게 맞다!

0.2074396

In [37]:
favorite_title = 'Shawshank Redemption, The (1994)'
title_id = title_to_idx[favorite_title]
similar_title = als_model.similar_items(title_id, N=15)
similar_title

[(157, 0.17804442),
 (121, 0.14116603),
 (23, 0.12969548),
 (222, 0.12782803),
 (51, 0.121067),
 (248, 0.11692512),
 (269, 0.11368449),
 (48, 0.092543684),
 (3628, 0.0886219),
 (233, 0.083652414),
 (323, 0.083128594),
 (3629, 0.07945285),
 (3517, 0.06835021),
 (100, 0.064839184),
 (110, 0.062024493)]

In [38]:
#title_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_title]

['Shawshank Redemption, The (1994)',
 'Silence of the Lambs, The (1991)',
 "Schindler's List (1993)",
 'Pulp Fiction (1994)',
 'Fargo (1996)',
 'Good Will Hunting (1997)',
 'GoodFellas (1990)',
 'Saving Private Ryan (1998)',
 '8 mile (2003)',
 'Usual Suspects, The (1995)',
 'Dead Man Walking (1995)',
 'Inception (2010)',
 'Paralyzing Fear: The Story of Polio in America, A (1998)',
 'Amadeus (1984)',
 'Groundhog Day (1993)']

In [39]:
def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

print("슝=3")

슝=3


In [40]:
get_similar_title('Shawshank Redemption, The (1994)')

['Shawshank Redemption, The (1994)',
 'Silence of the Lambs, The (1991)',
 "Schindler's List (1993)",
 'Pulp Fiction (1994)',
 'Fargo (1996)',
 'Good Will Hunting (1997)',
 'GoodFellas (1990)',
 'Saving Private Ryan (1998)',
 '8 mile (2003)',
 'Usual Suspects, The (1995)']

## 8) 내가 좋아할 만한 영화들을 추천받아보기

In [41]:
user = user_to_idx[6041]

In [43]:
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)

In [46]:
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in title_recommended]

["Schindler's List (1993)",
 'Fargo (1996)',
 'Pulp Fiction (1994)',
 'Saving Private Ryan (1998)',
 'GoodFellas (1990)',
 'Good Will Hunting (1997)',
 'American Beauty (1999)',
 'E.T. the Extra-Terrestrial (1982)',
 'Braveheart (1995)',
 'Sixth Sense, The (1999)',
 'Usual Suspects, The (1995)',
 'Raiders of the Lost Ark (1981)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'When Harry Met Sally... (1989)',
 'Christmas Story, A (1983)',
 'Amadeus (1984)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Groundhog Day (1993)',
 'L.A. Confidential (1997)',
 'Sling Blade (1996)']